In [1]:
import os
os.environ["JAX_ENABLE_X64"] = "true"
import sys
sys.path.append('..')
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import *
from filters import resamplers
from filters.filter import ParticleFilter
from filters.filter import ParticleFilterAll
from jax import config
jax.config.update("jax_enable_x64", True)
import numpy as np

float64


Initialisation of a twin experiment. 

In [2]:
signal_params = ConfigDict(KS_params_SALT)
ensemble_params = ConfigDict(KS_params_SALT)
ensemble_params.update(E=1,P=32,noise_magnitude = 0.01,stochastic_advection_basis='sin')
ensemble_params.update(E=128,P=32,noise_magnitude = 0.01,stochastic_advection_basis='sin')

Now we specify the models, by calling the class.

In [3]:
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

print(ensemble_model.params,ensemble_model.params.nt)
available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)

Advection_basis_name: sin
E: 128
Forcing_basis_name: none
P: 32
S: 0
c_0: 0
c_1: 1
c_2: 1
c_3: 0.0
c_4: 1
dt: 0.25
equation_name: Kuramoto-Sivashinsky
initial_condition: Kassam_Trefethen_KS_IC
method: Dealiased_SETDRK4
noise_magnitude: 0.01
nt: 600
nx: 256
stochastic_advection_basis: sin
tmax: 150
xmax: 100.53096491487338
xmin: 0
 600
multinomial, systematic, no_resampling, none, default


In [4]:
observation_spatial_frequency = 32
observation_locations = np.arange(0,signal_model.x.shape[0],observation_spatial_frequency)
observation_noise = 0.1
number_of_observations_time = 32 #ensemble_model.params.nt
observation_temporal_frequency = int(ensemble_model.params.nt/number_of_observations_time)
print(f"Observation temporal frequency: {observation_temporal_frequency}")
print(f"Observation spatial locations:{observation_locations}")
pf_systematic = ParticleFilterAll(
    n_particles = ensemble_params.E,
    n_steps = observation_temporal_frequency,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    resampling="systematic",#'default',
    observation_locations = observation_locations,
)

Observation temporal frequency: 18
Observation spatial locations:[  0  32  64  96 128 160 192 224]


To run the filter with all outputs, the input arrays need to have 3 dimensions.
They are (time, n_particles, space).

In [5]:
da_steps = number_of_observations_time
print(initial_ensemble.shape, initial_signal.shape, da_steps)
initial_ensemble = initial_ensemble[None, ...] # adding the time dimension
initial_signal = initial_signal[None, ...] # adding the time dimension
key = jax.random.PRNGKey(0) # random key for JAX
final, all = pf_systematic.run(initial_ensemble, initial_signal, da_steps,key) #the final input is scan length? 

(128, 256) (1, 256) 32


In [6]:
print(len(final), len(all))

3 3


The two outputs are tuples. The first tuple has length 2 and contains the final ensemble and the final signal arrays.
The second tuple has length three and contains all outputs through time of the ensemble, signal and observation.

In [7]:
for i in range(3):
    print(all[i].shape)
arr = all[0]
all_0_reshaped = jnp.reshape(arr, (-1, arr.shape[2], arr.shape[3]))
print(all_0_reshaped.shape)
def plot_member(e):
    plt.imshow(all_0_reshaped[:, e, :], aspect='auto', origin='lower')
    plt.title(f'Ensemble member {e}')
    plt.xlabel('Space')
    plt.ylabel('Time')
    plt.show()

interact(plot_member, e=(0, all_0_reshaped.shape[1] - 1))


(32, 18, 128, 256)
(32, 18, 1, 256)
(32, 1, 256)
(576, 128, 256)


interactive(children=(IntSlider(value=63, description='e', max=127), Output()), _dom_classes=('widget-interact…

<function __main__.plot_member(e)>

In [8]:
for i in range(2):
    print(final[i].shape)

(1, 128, 256)
(1, 1, 256)


Put in the initial condition.

In [9]:
print(initial_ensemble.shape)
print(all[0].shape)
all_0_reshaped = jnp.reshape(all[0], (-1, all[0].shape[2], all[0].shape[3]))
print(all_0_reshaped.shape)

particles =jnp.concatenate([initial_ensemble[...], all_0_reshaped], axis=0)
print(particles.shape)

print(all[1].shape)
all_1_reshaped = jnp.reshape(all[1], (-1, all[1].shape[2], all[1].shape[3]))
print(all_1_reshaped.shape)
print(initial_signal.shape)
signal = jnp.concatenate([initial_signal[...], all_1_reshaped], axis=0)
print(signal.shape)
print(all[2].shape)
# observations = jnp.concatenate([initial_signal[...], all[2]], axis=0)
observations = all[2][:,:, observation_locations]
print(observations.shape)


(1, 128, 256)
(32, 18, 128, 256)
(576, 128, 256)
(577, 128, 256)
(32, 18, 1, 256)
(576, 1, 256)
(1, 1, 256)
(577, 1, 256)
(32, 1, 256)
(32, 1, 8)


In [10]:
def plot(time):
    plt.plot(signal_model.x, signal[time,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[time,:,:].T, color='b',label='particles',linewidth=0.1)
    if (time ) % observation_temporal_frequency == 0 and time !=0:
        plt.plot(signal_model.x[observation_locations], observations[int(time/observation_temporal_frequency),0,:], 'ro',label='observations')
    plt.show()

interact(plot, time=(0, ensemble_model.params.nt - 1, 1))

interactive(children=(IntSlider(value=299, description='time', max=599), Output()), _dom_classes=('widget-inte…

<function __main__.plot(time)>

In [11]:
ensemble_params_none  = ConfigDict(KS_params_SALT)
ensemble_params_none.update(E=128,P=32,noise_magnitude = 0.01,stochastic_advection_basis='sin')
ensemble_model_none   = ETD_KT_CM_JAX_Vectorised(ensemble_params_none)
initial_signal_none   = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble_none = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

pf_none = ParticleFilterAll(
    n_particles = ensemble_params.E,
    n_steps = observation_temporal_frequency,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    resampling="no_resampling",#'default',
    observation_locations = observation_locations,
)
final_none, all_none = pf_none.run(initial_ensemble, initial_signal, da_steps,key) #the final input is scan length?
initial_ensemble_none = initial_ensemble_none[None, ...] # adding the time dimension
initial_signal_none = initial_signal_none[None, ...] # adding the time dimension
key = jax.random.PRNGKey(0) # random key for JAX

In [12]:
final_none, all_none = pf_none.run(initial_ensemble_none, initial_signal_none, da_steps,key)

In [13]:
all_none_0_reshaped = jnp.reshape(all_none[0], (-1, all_none[0].shape[2], all_none[0].shape[3]))
print(all_none_0_reshaped.shape)

def plot_member_none(e):
    plt.imshow(all_none_0_reshaped[:, e, :], aspect='auto', origin='lower')
    plt.title(f'Ensemble member {e} (no resampling)')
    plt.xlabel('Space')
    plt.ylabel('Time')
    plt.show()

interact(plot_member_none, e=(0, all_none_0_reshaped.shape[1] - 1))

(576, 128, 256)


interactive(children=(IntSlider(value=63, description='e', max=127), Output()), _dom_classes=('widget-interact…

<function __main__.plot_member_none(e)>

In [14]:
def plot(time):
    plt.plot(signal_model.x, signal[time,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[time,:,:].T, color='b',label='particles',linewidth=0.1)
    plt.plot(signal_model.x, all_none_0_reshaped[time,:,:].T, color='g',label='particles (no resampling)',linewidth=0.1)
    if (time ) % observation_temporal_frequency == 0 and time !=0:
        plt.plot(signal_model.x[observation_locations], observations[int(time/observation_temporal_frequency),0,:], 'ro',label='observations')
    plt.show()

interact(plot, time=(0, ensemble_model.params.nt - 1, 1))

interactive(children=(IntSlider(value=299, description='time', max=599), Output()), _dom_classes=('widget-inte…

<function __main__.plot(time)>

In [15]:
from metrics.ensemble import rmse, crps,crps_internal

# Compute RMSE and CRPS for the ensemble (with resampling)
rmse_ensemble = rmse(signal,particles)
crps_ensemble = crps_internal(signal, particles)#crps(signal,particles)

# Compute RMSE and CRPS for the ensemble (no resampling)
rmse_none = rmse(signal,all_none_0_reshaped)
#crps_none = crps(signal,all_none_0_reshaped)

print("RMSE (systematic):", rmse_ensemble)
print("CRPS (systematic):", crps_ensemble)
print("RMSE (no resampling):", rmse_none)
print("CRPS (no resampling):", crps_none)

NameError: name 'jax' is not defined

In [ ]:
dpi=300
plt.figure(figsize=(7, 4), dpi=dpi)
plt.plot(np.arange(da_steps + 1), rmse_ensemble, color='black', marker='o', label='Particle Filter')
plt.plot(np.arange(da_steps + 1), rmse_none, color='blue', marker='s', label='No Particle Filter')
plt.xlabel('Lead Time (DA step)')
plt.ylabel('RMSE')
plt.title('RMSE')
plt.legend()
plt.show()

plt.figure(figsize=(7, 4), dpi=dpi)
plt.plot(np.arange(da_steps + 1), crps_ensemble, color='black', marker='o', label='Particle Filter')
plt.plot(np.arange(da_steps + 1), crps_none, color='blue', marker='s', label='No Particle Filter')
plt.xlabel('Lead Time (DA step)')
plt.ylabel('Score')
plt.title('CRPS vs Lead Time')
plt.legend()
plt.show()